<a href="https://colab.research.google.com/github/Parth-DH/SigVerify/blob/master/V_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
cd var

/var


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ICDAR_DATA_1.zip to ICDAR_DATA_1.zip
User uploaded file "ICDAR_DATA_1.zip" with length 4681115 bytes


In [0]:
!unzip ICDAR_DATA_1.zip

Archive:  ICDAR_DATA_1.zip
   creating: ICDAR_Data/
   creating: ICDAR_Data/Train_Data/
   creating: ICDAR_Data/Train_Data/authentic/
   creating: ICDAR_Data/Train_Data/authentic/fifteen/
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_01.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_02.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_03.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_04.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_05.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_06.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_07.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_08.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_09.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_10.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_11.jpg  
  inflating: ICDAR_Data/Train_Data/authentic/fifteen/015_12.jpg  
  inflating: ICDAR_D

In [0]:
!pwd

/var


In [0]:
from keras.legacy import interfaces
import keras.backend as K
from keras.optimizers import Optimizer

class Modified_SGD(Optimizer):
  """ Modified Stochastic gradient descent optimizer.
  Almost all this class is Keras SGD class code. I just reorganized it
  in this class to allow layer-wise momentum and learning-rate
  Includes support for momentum,
  learning rate decay, and Nesterov momentum.
  Includes the possibility to add multipliers to different
  learning rates in each layer.
  # Arguments
  lr: float >= 0. Learning rate.
  momentum: float >= 0. Parameter updates momentum.
  decay: float >= 0. Learning rate decay over each update.
        nesterov: boolean. Whether to apply Nesterov momentum.
        lr_multipliers: dictionary with learning rate for a specific layer
        for example:
            # Setting the Learning rate multipliers
            LR_mult_dict = {}
            LR_mult_dict['c1']=1
            LR_mult_dict['c2']=1
            LR_mult_dict['d1']=2
            LR_mult_dict['d2']=2
        momentum_multipliers: dictionary with momentum for a specific layer 
        (similar to the lr_multipliers)
  """

  def __init__(self, lr=0.01, momentum=0., decay=0.,
                 nesterov=False, lr_multipliers=None, momentum_multipliers=None, **kwargs):
    super(Modified_SGD, self).__init__(**kwargs)
    with K.name_scope(self.__class__.__name__):
      self.iterations = K.variable(0, dtype='int64', name='iterations')
      self.lr = K.variable(lr, name='lr')
      self.momentum = K.variable(momentum, name='momentum')
      self.decay = K.variable(decay, name='decay')
    self.initial_decay = decay
    self.nesterov = nesterov
    self.lr_multipliers = lr_multipliers
    self.momentum_multipliers = momentum_multipliers

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
      grads = self.get_gradients(loss, params)
      self.updates = [K.update_add(self.iterations, 1)]

      lr = self.lr
      if self.initial_decay > 0:
        lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        
        
      # momentum
      shapes = [K.int_shape(p) for p in params]
      moments = [K.zeros(shape) for shape in shapes]
      self.weights = [self.iterations] + moments
      for p, g, m in zip(params, grads, moments):

        if self.lr_multipliers != None:
          if p.name in self.lr_multipliers:
            new_lr = lr * self.lr_multipliers[p.name]
          else:
            new_lr = lr
        else:
          new_lr = lr

        if self.momentum_multipliers != None:
          if p.name in self.momentum_multipliers:
            new_momentum = self.momentum * \
                        self.momentum_multipliers[p.name]
          else:
            new_momentum = self.momentum
        else:
          new_momentum = self.momentum

        v = new_momentum * m - new_lr * g  # velocity
        self.updates.append(K.update(m, v))

        if self.nesterov:
          new_p = p + new_momentum * v - new_lr * g
        else:
          new_p = p + v

        # Apply constraints.
        if getattr(p, 'constraint', None) is not None:
          ew_p = p.constraint(new_p)

        self.updates.append(K.update(p, new_p))
    return self.updates

  def get_config(self):
    config = {'lr': float(K.get_value(self.lr)),
                  'momentum': float(K.get_value(self.momentum)),
                  'decay': float(K.get_value(self.decay)),
                  'nesterov': self.nesterov,
                  'lr_multipliers': float(K.get_value(self.lr_multipliers)),
                  'momentum_multipliers': float(K.get_value(self.momentum_multipliers))}
    base_config = super(Modified_SGD, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [0]:
import random
class ICDAR_Loader:
  """class that loads and prepares the ICDAR Dataset"""
  def __init__(self, dataset_path, batch_size):
    self.dataset_path=dataset_path
    self.train_dictionary={}
    self.evaluation_dictionary={}
    self.image_width=200
    self.image_height=200
    self.batch_size=batch_size
    self.train=[]
    self.val=[]
    self.eval=[]
    self.cti=0
    self.cvi=0
    self.cei=0
    self.load_dataset()
  def load_dataset(self):
    train_path=os.path.join(self.dataset_path, 'Train_Data')
    validation_path=os.path.join(self.dataset_path, 'Eval')
    
    for sig in os.listdir(train_path):
      print(sig)
      sig_path=os.path.join(train_path, sig)
      
      curr_sig_dict={}
      for char in os.listdir(sig_path):
        char_path=os.path.join(sig_path, char)
        curr_sig_dict[char]=os.listdir(char_path)
      self.train_dictionary[sig]=curr_sig_dict
      
    for sig in os.listdir(validation_path):
      sig_path=os.path.join(validation_path, sig)
      
      curr_sig_dict={}
      for char in os.listdir(sig_path):
        char_path=os.path.join(sig_path, char)
        curr_sig_dict[char]=os.listdir(char_path)
      self.evaluation_dictionary[sig]=curr_sig_dict
     
  def split_train_dataset(self):
    """Splits the train set into training and validation"""
    available_sig = list(self.train_dictionary.keys())
    number_of_sig = len(available_sig)
    train_indexes = random.sample(range(0, number_of_sig -1), int(0.8*number_of_sig))
    train_indexes.sort(reverse=True)
    
    for index in train_indexes:
      self.train.append(available_sig[index])
      print(self.train)
      available_sig.pop(index)
    
    self.val=available_sig
    self.eval=list(self.evaluation_dictionary.keys())
    
  def _convert_path_list_to_image_and_labels(self, path_list, is_one_shot_task):
    """loads image and correstponding lables from the path"""
    number_of_pairs = int(len(path_list)/2)
    pairs_of_images = [np.zeros((number_of_pairs, self.image.height, 1)) for i in range(2)]
    labels=np.zeros((number_of_pairs, 1))
    
    for pair in range(number_of_pairs):
      image=Image.open(path_list[pair *2])
      image = np.asarray(image).astype(np.float64)
      image = image/ image.std() - image.mean()
      
      pairs_of_images[0][pair, :, :, 0] = image
      image = Image.open(path_list[pair *2 +1])
      image = np.asarray(image).astype(float64)
      image = image / image.std() - image.mean()
      
      pairs_of_images[1][pair, :, :, 0] = image
      
      if not is_one_shot_task:
        if(pair+1)%2==0:
          labels[pair] = 0
        else:
          labels[pair] = 1
      else:
        if pair == 0:
          labels[pair]=1
        else:
          labels[pair]=0
   
    if not is_one_shot_task:
      random_permutation = np.random.permutation(number_of_pairs)
      labels = labels[random_permutation]
      pairs_of_images[0][:, :, :, :] = pairs_of_images[0][random_permutation, :, :, :]
      pairs_of_images[1][:, :, :, :] = pairs_of_images[1][random_permutation, :, :, :]

    return pairs_of_images, labels

  def get_train_batch(self):
    print(self.train)
    print(self.cti)
    curr_sig=self.train[self.cti]
    avail_chars = list(self.train_dictionary[curr_sig].keys())
    number_of_chars=len(avail_chars)
  
    batch_image_path=[]
  
    selected_chars_indexes = [random.randit(0, number_of_chars-1) for i in range(self.batch_size)]
    for index in selected_chars_indexes:
      current_char = avail_chars[index]
      available_images = (self.train_dictionary[curr_sig])[curr_char]
    
      image_path= os.path.join(self.dataset_path, 'Train_Data', curr_sig, curr_char)
      image_indexes = random.sample(range(0, 10), 3)
      image = os.path.join(image_path, available_images[image_indexes[0]])
      batch_images_path.append(image)
      image= os.path.join(image_path, available_images[image_indexes[1]])
      batch_images_path.append(image)
    
      """pairs from different sigs"""
      image = os.path.join(image_path, available_images[image_indexes[2]])
      bacth_images_path.append(image)
      different_characters = avail_chars[:]
      different_characters.pop(index)
      different_character_index = random.sample(range(0, number_of_chars - 1), 1)
      curr_char = different_characters[different_character_index[0]]
      available_images = (self.train_dictionary[curr_sig])[curr_char]
      image_indexes = random.sample(range(0, 20), 1)
      image_path = os.path.join(self.dataset_path, 'Train_Daata', curr_sig, curr_char)
      image = os.path.join(image_path, available_images[image_indexes[0]])
      batch_images_path.append(image)
  
    self.cti+=1
    if(self.cti >14):
      self.cti = 0
    images, labels = self._convert_path_list_to_image_and_labels(batch_images_path, is_one_shot_task=False)
  
    return images, labels

  def get_one_shot_batch(self, support_set_size, is_validation):
    '''loads and returns batch of one shot learning task images'''
    if is_validation:
      chars=self.val
      curr_char_index= self.cvi
      image_folder_name = 'Train_Data'
      dictionary=self.train_dictionary
    else:
      chars=self.eval
      curr_char_index=self.cei
      image_folder_name='images_eval'
      dictionary=self.evaluation_dictionary
    curr_char= chars[curr_char_index]
    avail_chars = list(dictionary[curr_char].keys())
    number_of_characters = len(avail_chars)
  
    batch_images_path = []
  
    test_char_index=random.sample(range(0, number_of_characters), 1)
    #get test image
    curr_char=avail_char[test_char_index[0]]
    available_images = (dictionary[curr_sig])[curr_char]
    image_indexes = random.sample(range(0, 20), 2)
    image_path=os.path.join(self.dataset_path, image_folder_name, curr_sig, curr_char)
  
    test_image= os.path.join(image_path, available_images[image_indexes[0]])
    batch_images_path.append(test_image)
    image=os.path.join(image_path, available_images[image_indexes[1]])
    batch_images_path.append(image)
  
    if support_test_size== -1:
      number_of_support_characters=number_of_characters
    else:
      number_of_support_characters= support_set_size
   
    different_characters = available_characters[:]
    different_characters.pop(test_character_index[0])
  
    if number_of_characters < number_of_support_characters:
      number_of_support_characters = number_of_characters
    support_characters_indexes = random.sample(range(0, number_of_characters-1), number_of_support_characters -1)
    for index in support_characters_indexes:
      curr_char= different_characters[index]
      available_images=(dictionary[curr_sig])[curr_char]
      image_path = os.path.join(self.dataset_path, image_folder_name, curr_sig, curr_char)
      image_indexes=random.sample(range(0, 20), 1)
      image = os.path.join(image_path, available_images[image_indexes[0]])
      batch_images_path.append(test_image)
      batch_images_path.append(image)
    images, labels = self._convert_path_list_to_image_and_labels(batch_images_path, is_one_shot_task = True)
  
    return images, labels
  
  def one_shot_test(self, support_set_size, number_of_tasks_per_sig, is_validation):
    if is_validation:
      sigs= self.val
      print('\nMaking One Shot Task on validation alphabets:')
    else:
      sigs=self.eval
      print('\nMaking One Shot Task on evaluation alphabets:')
    mean_global_accuracy = 0
    
    for sig in sigs:
      mean_alphabet_accuracy=0
      for _ in range(number_of_tasks_per_sig):
        images, _= self.get_one_shot_batch(support_set_size, is_validation=is_validation)
        probabilities=model.predict_on_batch(images)
        
        if np.argmax(probabilities)==0 and probabilities.std()>0.01:
          accuracy=1.0
        else:
          accuracy=0.0
          
        mean_sig_accuracy+=accuracy
        mean_global_accuracy+= accuracy
      mean_sig_accuracy /= number_of_tasks_per_sig
      print(sig + 'signature' + ', accuracy: ' + str(mean_sig_accuracy))
      if is_validation:
        self.cvi+=1
      else:
        self.cei+=1
    mean_global_accuracy /= (len(sigs)*number_of_tasks_per_sig)
    print('\nMean global accuracy: ' + str(mean_global_accuracy))
    
    if is_validation:
      self.cvi=0
    else:
      self.cei=0
    return mean_global_accuracy
    

    

In [0]:
import os

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, Subtract, Lambda
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
import keras.backend as K

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np


In [0]:
class SiameseNetwork:
  """construct SNET for training"""
  def __init__(self, dataset_path, learning_rate, batch_size, learning_rate_multipliers, l2_regularization_penalization, tensorboard_log_path):
    self.input_shape = (200, 200, 1)
    self.model = []
    self.learning_rate = learning_rate
    self.ICDAR_loader = ICDAR_Loader(dataset_path=dataset_path, batch_size=batch_size)
    self.summary_writer = tf.summary.FileWriter(tensorboard_log_path)
    self.learning_rate_multipliers=learning_rate_multipliers
    self.l2_regularization_penalization=l2_regularization_penalization
    self.tensorboard_log_path=tensorboard_log_path
  def __construct_siamese_architecture(self):
    convolutional_net = Sequential()
    convolutional_net.add(Conv2D(filters=64, kernel_size=(10, 10),
                                     activation='relu',
                                     input_shape=self.input_shape,
                                     kernel_regularizer=l2(
                                         l2_regularization_penalization['Conv1']),
                                     name='Conv1'))
    convolutional_net.add(MaxPool2D())

    convolutional_net.add(Conv2D(filters=128, kernel_size=(7, 7),
                                     activation='relu',
                                     kernel_regularizer=l2(
                                         l2_regularization_penalization['Conv2']),
                                     name='Conv2'))
    convolutional_net.add(MaxPool2D())

    convolutional_net.add(Conv2D(filters=128, kernel_size=(4, 4),
                                     activation='relu',
                                     kernel_regularizer=l2(
                                         l2_regularization_penalization['Conv3']),
                                     name='Conv3'))
    convolutional_net.add(MaxPool2D())

    convolutional_net.add(Conv2D(filters=256, kernel_size=(4, 4),
                                     activation='relu',
                                     kernel_regularizer=l2(
                                         l2_regularization_penalization['Conv4']),
                                     name='Conv4'))
    convolutional_net.add(MaxPool2D())

    convolutional_net.add(Flatten())
    convolutional_net.add(
            Dense(units=4096, activation='sigmoid',
                  kernel_regularizer=l2(
                      l2_regularization_penalization['Dense1']),
                  name='Dense1'))

        # Now the pairs of images
    input_image_1 = Input(self.input_shape)
    input_image_2 = Input(self.input_shape)
  
    encoded_image_1 = convolutional_net(input_image_1)
    encoded_image_2 = convolutional_net(input_image_2)

        # L1 distance layer between the two encoded outputs
        # One could use Subtract from Keras, but we want the absolute value
    l1_distance_layer = Lambda(
              lambda tensors: K.abs(tensors[0] - tensors[1]))
    l1_distance = l1_distance_layer([encoded_image_1, encoded_image_2])

        # Same class or not prediction
    prediction = Dense(units=1, activation='sigmoid')(l1_distance)
    self.model = Model(
            inputs=[input_image_1, input_image_2], outputs=prediction)

        # Define the optimizer and compile the model
    optimizer = Modified_SGD(
            lr=self.learning_rate,
            lr_multipliers=self.learning_rate_multipliers,
            momentum=0.5)

    self.model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'],
                           optimizer=optimizer)
  def __write_logs_to_tensorboard(self, current_iteration, train_losses, train_accuracies, validation_accuracy, evaluate_each):
    summary = tf.summary()
    
    for index in range(0, evaluate_each):
      value = summary.value.add()
      value.simple_value=train_losses[index]
      value.tag = 'Train Loss'

      value = summary.value.add()
      value.simple_value = train_accuracies[index]
      value.tag = 'Train Accuracy'

      if index == (evaluate_each - 1):
        value = summary.value.add()
        value.simple_value = validation_accuracy
        value.tag = 'One-Shot Validation Accuracy'

      self.summary_writer.add_summary(
            summary, current_iteration - evaluate_each + index + 1)
      self.summary_writer.flush()
  def train_siamese_network(self, number_of_iterations, support_set_size, final_momentum, momentum_slope, evaluate_each, model_name):
    self.ICDAR_loader.split_train_dataset()
    train_losses=np.zeros(shape=(evaluate_each))
    train_accuracies= np.zeros(shape=(evaluate_each))
    count = 0
    earrly_stop = 0
    best_validation_accuracy=0.0
    best_accuracy_iteration=0.0
    validation_accuracy=0.0
    
    #train loop
    for iteration in range(number_of_iterations):
      #train set
      images, labels= self.ICDAR_loader.get_train_batch()
      train_loss, train_accuracy=self.model.train_on_batch(images, labels)
      # Decay learning rate 1 % per 500 iterations (in the paper the decay is
      # 1% per epoch). Also update linearly the momentum (starting from 0.5 to 1)
      if (iteration +1)%500==0:
        K.set_value(self.model.optimizer.lr, K.get_value(
                    self.model.optimizer.lr) * 0.99)
      if K.get_value(self.model.optimizer.momentum) < final_momentum:
        K.set_value(self.model.optimizer.momentum, K.get_value(
        self.model.optimizer.momentum) + momentum_slope)
      train_losses[count] = train_loss
      train_accuracies[count] = train_accuracy
      
      #validation set
      count += 1
      print('Iteration %d/%d: Train loss: %f, Train Accuracy: %f, lr = %f' %
      (iteration + 1, number_of_iterations, train_loss, train_accuracy, K.get_value(
                      self.model.optimizer.lr)))
      # Each 100 iterations perform a one_shot_task and write to tensorboard the
      # stored losses and accuracies
      if (iteration + 1) % evaluate_each == 0:
        number_of_runs_per_alphabet = 40
        # use a support set size equal to the number of character in the alphabet
        validation_accuracy = self.omniglot_loader.one_shot_test(
                    self.model, support_set_size, number_of_runs_per_alphabet, is_validation=True)

        self.__write_logs_to_tensorboard(
                    iteration, train_losses, train_accuracies,
                    validation_accuracy, evaluate_each)
        count = 0
        # Some hyperparameters lead to 100%, although the output is almost the same in 
        # all images. 
        if (validation_accuracy == 1.0 and train_accuracy == 0.5):
          print('Early Stopping: Gradient Explosion')
          print('Validation Accuracy = ' +
                          str(best_validation_accuracy))
          return 0
        elif train_accuracy == 0.0:
          return 0
        else:
          # Save the model
          if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_accuracy_iteration = iteration
                        
            model_json = self.model.to_json()

            if not os.path.exists('./models'):
              os.makedirs('./models')
            with open('models/' + model_name + '.json', "w") as json_file:
              json_file.write(model_json)
            self.model.save_weights('models/' + model_name + '.h5')
      if iteration - best_accuracy_iteration > 10000:
        print('Early Stopping: validation accuracy did not increase for 10000 iterations')
        print('Best Validation Accuracy = ' + str(best_validation_accuracy))
        print('Validation Accuracy = ' + str(best_validation_accuracy))
        break  
    print('Trained Ended!')
    return best_validation_accuracy

In [0]:
def main():
    dataset_path = 'var/ICDAR_Data'
    learning_rate = 10e-4
    batch_size = 32
    # Learning Rate multipliers for each layer
    learning_rate_multipliers = {}
    learning_rate_multipliers['Conv1'] = 1
    learning_rate_multipliers['Conv2'] = 1
    learning_rate_multipliers['Conv3'] = 1
    learning_rate_multipliers['Conv4'] = 1
    learning_rate_multipliers['Dense1'] = 1
    # l2-regularization penalization for each layer
    l2_penalization = {}
    l2_penalization['Conv1'] = 1e-2
    l2_penalization['Conv2'] = 1e-2
    l2_penalization['Conv3'] = 1e-2
    l2_penalization['Conv4'] = 1e-2
    l2_penalization['Dense1'] = 1e-4
    # Path where the logs will be saved
    tensorboard_log_path = './logs/siamese_net_lr10e-4'
    siamese_network = SiameseNetwork(
        dataset_path=dataset_path,
        learning_rate=learning_rate,
        batch_size=batch_size,
        learning_rate_multipliers=learning_rate_multipliers,
        l2_regularization_penalization=l2_penalization,
        tensorboard_log_path=tensorboard_log_path
    )
    # Final layer-wise momentum (mu_j in the paper)
    momentum = 0.9
    # linear epoch slope evolution
    momentum_slope = 0.01
    support_set_size = 20
    evaluate_each = 1000
    number_of_train_iterations = 1000000

    validation_accuracy = siamese_network.train_siamese_network(number_of_iterations=number_of_train_iterations,
                                                                support_set_size=support_set_size,
                                                                final_momentum=momentum,
                                                                momentum_slope=momentum_slope,
                                                                evaluate_each=evaluate_each, 
                                                                model_name='siamese_net_lr10e-4')
    if validation_accuracy == 0:
        evaluation_accuracy = 0
    else:
        # Load the weights with best validation accuracy
        siamese_network.model.load_weights('.models/siamese_net_lr10e-4.h5')
        evaluation_accuracy = siamese_network.omniglot_loader.one_shot_test(siamese_network.model,
                                                                        20, 40, False)
    
    print('Final Evaluation Accuracy = ' + str(evaluation_accuracy))


if __name__ == "__main__":
    main()
    cwd=os.getcwd()
    print(os.listdir(cwd))

FileNotFoundError: ignored

In [0]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# New Section